# Env

In [21]:
import requests
import numpy as np

In [22]:
url = "http://127.0.0.1:8800"

# Mobile Base Control

In [43]:
payload = {
    "action": {
        "type": "run_code",
        "payload": {
            "code": """
pos = get_mobile_joint_position()
RESULT = {
    "mobile_joint_position": pos
}
"""
        },
    }
}
response = requests.post(f"{url}/send_action", json=payload)
response.json()

{'status': 'success',
 'result': {'mobile_joint_position': [-0.7689915895276425,
   0.04751390092909531,
   5.4956611614618276e-06]}}

In [44]:
payload = {
    "action": {
        "type": "run_code",
        "payload": {
            "code": """
result = set_mobile_target_joint([-1, 0, 0])
RESULT = result
"""
        },
    }
}
response = requests.post(f"{url}/send_action", json=payload)
response.json()

KeyboardInterrupt: 

In [29]:
payload = {
    "action": {
        "type": "run_code",
        "payload": {
            "code": """
result = set_mobile_target_joint([0, 0, 0])
RESULT = result
"""
        },
    }
}
response = requests.post(f"{url}/send_action", json=payload)
response.json()

{'status': 'success', 'result': False}

# Arm Joint Control

In [45]:
payload = {
    "action": {
        "type": "run_code",
        "payload": {
            "code": """
pos = get_arm_joint_position()
RESULT = {
    "arm_joint_position": pos
}
"""
        },
    }
}
response = requests.post(f"{url}/send_action", json=payload)
arm_joint_position = response.json()["result"]["arm_joint_position"]
arm_joint_position

[-0.011633103160825166,
 -1.0324668417865355,
 0.05100330704507193,
 -2.2597091358244374,
 0.0630746969100158,
 1.5231123224146244,
 0.6766870461553404]

In [9]:
payload = {
    "action": {
        "type": "run_code",
        "payload": {
            "code": """
result = set_arm_target_joint([-0.011, -1.032, 0.051, -2.260, 0.063, 1.523, 0.675])
RESULT = result
"""
        },
    }
}
response = requests.post(f"{url}/send_action", json=payload)
response.json()

{'status': 'success', 'result': True}

# End Effector Control

In [10]:
payload = {
    "action": {
        "type": "run_code",
        "payload": {
            "code": """
pos, ori = get_ee_position()
RESULT = {
    "ee_position": {
        "pos": pos,
        "ori": ori
    }
}
"""
        },
    }
}
response = requests.post(f"{url}/send_action", json=payload)
ee_position = response.json()["result"]["ee_position"]
ee_position

{'pos': [-1.0406287249556778, -0.42676215006925666, 1.3018532821210027],
 'ori': [2.8494238286049303, 0.01706654765763549, 0.45690766151852813]}

In [11]:
ee_target_position = np.array(ee_position["pos"]) - np.array([0.1, 0.0, 0.0])
payload = {
    "action": {
        "type": "run_code",
        "payload": {
            "code": f"""
result = set_ee_target_position(({ee_target_position.tolist()}))
RESULT = result
"""
        },
    }
}
response = requests.post(f"{url}/send_action", json=payload)
response

<Response [200]>

# Pick and Place

In [ ]:
response = requests.get(f"{url}/env")


In [16]:
from requests import Response


payload = {
    "action": {
        "type": "run_code",
        "payload": {
            "code": """
objects = get_object_positions()
RESULT = {
    "objects": objects
}
"""
        },
    }
}
response: Response = requests.post(f"{url}/send_action", json=payload)
objects = response.json()["result"]["objects"]
objects

{'object_fork_0': {'id': 208,
  'pos': [-0.2999999999988103, -0.6499999999995241, 0.9397844891595659],
  'ori': [-4.7811028353352164e-17, 0.0, 5.203047773278387e-24]},
 'object_bowl_0': {'id': 209,
  'pos': [-2.199999970949615, -0.050000033383861386, 0.9624646907541051],
  'ori': [1.8517332434221947e-06,
   8.985393271654374e-07,
   -1.779374768979138e-08]}}

In [18]:
payload = {
    "action": {
        "type": "run_code",
        "payload": {
            "code": """
pos, ori = get_ee_position()
RESULT = {
    "ee_position": {
        "pos": pos,
        "ori": ori
    }
}
"""
        },
    }
}
response = requests.post(f"{url}/send_action", json=payload)
ee_position = response.json()["result"]["ee_position"]
ee_position

{'pos': [-1.667916250658859, -0.02848919751099643, 1.3029101015491613],
 'ori': [2.8468164683124533, 0.01612907427778132, -1.4301127838650836]}

In [19]:
objects["object_fork_0"]

{'id': 208,
 'pos': [-0.2999999999988103, -0.6499999999995241, 0.9397844891595659],
 'ori': [-4.7811028353352164e-17, 0.0, 5.203047773278387e-24]}

In [20]:
ee_target_position_1 = np.array(objects["object_fork_0"]["pos"]) + np.array(
    [0.0, 0.0, 0.1]
)
ee_target_position_2 = np.array(objects["object_fork_0"]["pos"]) + np.array(
    [0.0, 0.0, 0.0]
)
ee_target_position_3 = np.array(objects["object_fork_0"]["pos"]) + np.array(
    [0.0, 0.0, 0.2]
)

payload = {
    "action": {
        "type": "run_code",
        "payload": {
            "code": f"""
result = set_ee_target_position(({ee_target_position_1.tolist()}))
if result:
    result = set_ee_target_position(({ee_target_position_2.tolist()}))
if result:
    result = set_target_gripper_width({0.0})
if result:
    result = set_ee_target_position(({ee_target_position_3.tolist()}))
RESULT = result
"""
        },
    }
}
response = requests.post(f"{url}/send_action", json=payload)
response.json()

{'status': 'success', 'result': False}